# Double Node

In [90]:
class StateDoubleNode:
    def __init__(self, leftNode, rightNode, leftLabel, rightLabel, stateName):
        self.leftNode = leftNode
        self.rightNode = rightNode
        self.leftLabel = leftLabel
        self.rightLabel = rightLabel
        self.stateName = stateName
        
    def connectToLeft(self, node, label):
        self.leftNode = node
        self.leftLabel = label
        
    def connectToRight(self, node, label):
        self.rightNode = node
        self.rightLabel = label

    def setVariableValue(self, variableName, value):
        if variableName in vars(self).keys():
            vars(self)[variableName] = value

# State Graph Class

In [91]:
class StateGraph:
    def __init__(self, initialState, finalState):
        self.initialState = initialState
        self.finalState = finalState

    def setVariableValue(self, variableName, value):
        if variableName in vars(self).keys():
            vars(self)[variableName] = value

# Operations that create State Graphs

In [92]:
class StateGraphOperations:
    
    def __init__(self, nullSequenceSymbol, emptySequenceSymbol):
        self.nullSequenceSymbol = nullSequenceSymbol
        self.emptySequenceSymbol = emptySequenceSymbol
        
    def emptySequenceStateGraph(self):
        finalState = StateDoubleNode(None, None, None, None, None)
        initialState = StateDoubleNode(finalState, None, self.emptySequenceSymbol, None,
                                       None)
        return StateGraph(initialState, finalState)
    
    def nullSequenceStateGraph(self):
        finalState = StateDoubleNode(None, None, None, None, None)
        initialState = StateDoubleNode(None, None, None, None, None)
        return StateGraph(initialState, finalState)
    
    def symbolSequenceStateGraph(self, symbol):
        finalState = StateDoubleNode(None, None, None, None, None)
        initialState = StateDoubleNode(finalState, None, symbol, None, None)
        return StateGraph(initialState, finalState)
    
    def unionStateGraph(self, graphR, graphS):
        finalState = StateDoubleNode(None, None, None, None, None)
        initialState = StateDoubleNode(graphR.initialState, graphS.initialState,
                                       self.emptySequenceSymbol, self.emptySequenceSymbol, None)
        graphR.finalState.connectToLeft(finalState, self.emptySequenceSymbol)
        graphS.finalState.connectToLeft(finalState, self.emptySequenceSymbol)
        return StateGraph(initialState, finalState)
    
    def concatenationStateGraph(self, graphR, graphS):
        graphR.finalState.connectToLeft(graphS.initialState, self.emptySequenceSymbol)
        return StateGraph(graphR.initialState, graphS.finalState)
    
    def closureAsteriskStateGraph(self, graphR):
        finalState = StateDoubleNode(None, None, None, None, None)
        initialState = StateDoubleNode(graphR.initialState, finalState,
                                       self.emptySequenceSymbol, self.emptySequenceSymbol, None)
        graphR.finalState.connectToLeft(graphR.initialState, self.emptySequenceSymbol)
        graphR.finalState.connectToRight(finalState, self.emptySequenceSymbol)
        return StateGraph(initialState, finalState)
        
    def closurePlusStateGraph(self, graphR):
        finalState = StateDoubleNode(None, None, None, None, None)
        initialState = StateDoubleNode(graphR.initialState, None,
                                       self.emptySequenceSymbol, None, None)
        graphR.finalState.connectToLeft(graphR.initialState, self.emptySequenceSymbol)
        graphR.finalState.connectToRight(finalState, self.emptySequenceSymbol)
        return StateGraph(initialState, finalState)

# Util functions

In [93]:
from toolz import curry

@curry
def isOperator(operators, symbol):
    return symbol in operators

@curry
def isUnary(unaryOperators, operator):
    return operator in unaryOperators

@curry
def isBinary(binaryOperators, operator):
    return operator in binaryOperators

@curry
def isEmptySequence(emptySequence, symbol):
    return symbol == emptySequence

@curry
def isNullSequence(nullSequence, sequence):
    return sequence == nullSequence

@curry
def isEndOfSequence(endSequence, symbol):
    return symbol == endSequence

@curry
def isSymbolSequence(emptySequence, nullSequence, operators, sequence):
    return not (isEmptySequence(emptySequence)(sequence) or
                isNullSequence(nullSequence)(sequence) or
                isOperator(operators)(sequence))
@curry
def isAtomicSequence(emptySequence, nullSequence, operators, sequence):
    return (isEmptySequence(emptySequence)(sequence) or
            isNullSequence(nullSequence)(sequence) or
            isSymbolSequence(emptySequence)(nullSequence)(operators)(sequence))

def combineToRegularEx(dividedRegEx):
    if dividedRegEx[1] == None and dividedRegEx[2] == None:
        return dividedRegEx[0]
    if dividedRegEx[2] == None:
        return dividedRegEx[1] + dividedRegEx[0]
    return dividedRegEx[1] + dividedRegEx[0] + dividedRegEx[2]

@curry
def getStateGraphOperation(kettleOp, crossOp, unionOp,
    concatOp, emptySequence, nullSequence, graphOperations, symbol):  
    if symbol == kettleOp:
        return graphOperations.closureAsteriskStateGraph
    if symbol == crossOp:
        return graphOperations.closurePlusStateGraph
    if symbol == unionOp:
        return graphOperations.unionStateGraph
    if symbol == concatOp:
        return graphOperations.concatenationStateGraph
    if symbol == emptySequence:
        return graphOperations.emptySequenceStateGraph
    if symbol == nullSequence:
        return graphOperations.nullSequenceStateGraph
    return graphOperations.symbolSequenceStateGraph

def topElement(_list):
    if len(_list) == 0:
        return None
    return _list[len(_list)-1]

def listIsEmpty(_list):
    return len(_list) == 0

@curry
def countOpenParenthesis(openPar, symbols):
    counterOpenPar = 0
    for element in symbols:
        if element == openPar:
            counterOpenPar +=1
    return counterOpenPar

@curry
def countCloseParenthesis(closePar, symbols):
    counterClosePar = 0
    for element in symbols:
        if element == closePar:
            counterClosePar += 1
    return counterClosePar

@curry
def equalNumOpenParClosePar(counterClosePar, counterOpenPar, symbols):
    return counterClosePar(symbols) == counterOpenPar(symbols)

@curry
def getWeigth(closePar, unionOp, concatOp, kettleOp,
    crossOp, openPar, operator):
    cases = {closePar: 0, unionOp: 1, concatOp: 2, kettleOp: 3,
             crossOp: 3, openPar: 4}
    return cases[operator]

@curry
def hasGreaterEqWeigth(weightF, operatorR, operatorS):
    return weightF(operatorR) >= weightF(operatorS)

@curry
def topElementIsOpenPar(openPar, _list):
     return openPar == topElement(_list)
    
@curry
def accumulate(openPar, symbols, operators):
    accumulation = symbols.copy()
    operatorsLeft = operators.copy()
    while not listIsEmpty(operatorsLeft):
        operator = operatorsLeft.pop()
        if operator == openPar:
            break
        else:
            symbolj = ""
            symbolk = ""
            if isAUnaryOp(operator):
                symbolj = accumulation.pop()
                accumulation.append(operator + " " + symbolj)
            else:
                symbolk = accumulation.pop()
                symbolj = accumulation.pop()
                accumulation.append(operator + " " + symbolj + " " + symbolk)
    return (accumulation, operatorsLeft)

def getAcceptanceStatesFromDFAList(DFAList):
    return list(filter(lambda x: x[1] == True, DFAList))

def getEdgesFromDFAList(DFAList):
    edges = []
    for dfaNode in DFAList:
        for element in dfaNode[0].dfaNodeList:
            edges.append((dfaNode[0].nodeName, *element))
    return edges

def getAcceptanceStatesNameList(acceptanceStatesList):
    names = []
    for element in acceptanceStatesList:
        names.append(element[0].nodeName)
    return names

def reduceStateGraphToList(stateNode):
    currentVisitedNodes = []
    currentVisitedNodes.append(stateNode)
    for element in currentVisitedNodes:
        if element.leftNode != None and not element.leftNode in currentVisitedNodes:
            currentVisitedNodes.append(element.leftNode)
        if element.rightNode != None and not element.rightNode in currentVisitedNodes:
            currentVisitedNodes.append(element.rightNode)
    return currentVisitedNodes

def getEdgeList(nodeList):
    edgeList = []
    for node in nodeList:
        dfaNodesList = node[0].dfaNodeList
        nodeName = node[0].nodeName
        for element in dfaNodesList:
            edgeList.append((nodeName, *element))
    return edgeList

@curry
def nameStateGraphNodesList(letter, nodeList):
    nodeNumber = 0
    nodesL = nodeList.copy()
    for index in range(0, len(nodesL)):
        nodesL[index].setVariableValue("stateName", letter + str(index))
    return nodesL

# Symbols, Operators and other Variables

In [94]:
emptySequence = "$"
nullSequence = "¬"
endOfSequence = "!"
unionOp = "|"
concatOp = "."
kettleOp = "*"
crossOp = "+"
openPar = "("
closePar = ")"
operators = [openPar, closePar, unionOp, concatOp, kettleOp, crossOp]
unaryOperators = [kettleOp, crossOp]
binaryOperators = [unionOp, concatOp]
graphOperations = StateGraphOperations(nullSequence, emptySequence)
letter = "q"
startStr = "{" 
endStr = "}" 
separator = " "

# Specific Functions

In [95]:
isAnOperator = isOperator(operators)
isAUnaryOp = isUnary(unaryOperators)
isABinaryOp = isBinary(binaryOperators)
isAnEmptySequence = isEmptySequence(emptySequence)
isANullSequence = isNullSequence(nullSequence)
isTheEndOfSequence = isEndOfSequence(endOfSequence)
isASymbolSequence = isSymbolSequence(emptySequence)(nullSequence)(operators)
isAnAtomicSequence = isAtomicSequence(emptySequence)(nullSequence)(operators)
stateGraphOp = getStateGraphOperation(kettleOp)(crossOp)(unionOp)(concatOp)(emptySequence)(nullSequence)(graphOperations)
NCloseParEqualMOpenPar = equalNumOpenParClosePar(countCloseParenthesis(closePar))(countOpenParenthesis(openPar))
weightFOperators = getWeigth(closePar)(unionOp)(concatOp)(kettleOp)(crossOp)(openPar)
RhasGreaterWeightThanS = hasGreaterEqWeigth(weightFOperators)
accumulateExpression = accumulate(openPar)
nameNDFANodesList = nameStateGraphNodesList(letter)

# Regex Validator

In [96]:
def theRegExIsValid(string):
    symbols = []
    valid = True
    if len(string) == 0:
        return not valid
    if (isAUnaryOp(string[0]) or isABinaryOp(string[0]) 
        or isTheEndOfSequence(string[0]) or string[0] == closePar):
        return not valid
    if not endOfSequence in string:
        return not valid
    symbols.append(string[0])
    regEx = string[1:]
    index = 1
    for symbol in regEx:
        if isTheEndOfSequence(symbol):
            break
        if not isAnOperator(symbol):
            if topElement(symbols) == openPar or isABinaryOp(topElement(symbols)):
                symbols.append(symbol)
            else:
                symbols[len(symbols)-1] = symbols[len(symbols)-1] + symbol
        else:
            if isAUnaryOp(symbol):
                if (topElement(symbols) == closePar or
                    not isAnOperator(topElement(symbols)) or
                   isAUnaryOp(topElement(symbols))):
                    symbols.append(symbol)
                else:
                    return not valid
            if isABinaryOp(symbol):
                if (topElement(symbols) == closePar or
                    not isAnOperator(topElement(symbols)) or
                   isAUnaryOp(topElement(symbols))):
                    symbols.append(symbol)
                else:
                    return not valid
            if symbol == closePar:
                if topElement(symbols) == openPar or isABinaryOp(topElement(symbols)):
                    return not valid
                if countOpenParenthesis(openPar)(symbols) >= countCloseParenthesis(closePar)(symbols)+1:
                    symbols.append(symbol)
                else:
                    return not valid
            if symbol == openPar:
                if topElement(symbols) == openPar or isABinaryOp(topElement(symbols)):
                    symbols.append(symbol)
                else:
                    return not valid
        index +=1
    if not NCloseParEqualMOpenPar(symbols):
        return not valid
    if index == len(string)-1:
        return valid
    return not valid

# Conversor From Infix to Prefix

In [97]:
from toolz import curry

@curry
def fromInfixToPrefix(openPar, closePar, regExR):
    if len(regExR) == 1:
        return regExR
    symbols = []
    operators = []
    flag = False
    for symbol in regExR:
        if not isAnOperator(symbol):
            if not flag:
                symbols.append(symbol)
                flag = True
            else:
                symbols[len(symbols)-1] = symbols[len(symbols)-1] + symbol
        else:
            flag = False
            if symbol == closePar:
                accumulated = accumulateExpression(symbols)(operators)
                symbols = accumulated[0]
                operators = accumulated[1]
            else:
                if (listIsEmpty(operators) or topElementIsOpenPar(openPar)(operators) or
                    RhasGreaterWeightThanS(symbol)(topElement(operators))) :
                    operators.append(symbol)
                else:
                    accumulated = accumulateExpression(symbols)(operators)
                    symbols = accumulated[0]
                    operators = accumulated[1]
                    if symbol != openPar:
                        operators.append(symbol)
    while not listIsEmpty(operators):
        accumulated = accumulateExpression(symbols)(operators)
        symbols = accumulated[0]
        operators = accumulated[1]
    return symbols[0]
print(fromInfixToPrefix(openPar)(closePar)("(GO|GOTO|TOO|ON)*.ON.TOO"))
print(fromInfixToPrefix(openPar)(closePar)("(0|1.0*.1)*.0*"))

. * | GO | GOTO | TOO ON . ON TOO
. * . | 0 . 1 * 0 1 * 0


# Regex Divide Function

In [98]:
def dividePrefixRegEx(prefixRegEx):
    operators = []
    n_tokens = []
    divided_expression = ()
    if not isAnOperator(prefixRegEx[0]):
        return (prefixRegEx, None, None)
    for index in range(0, len(prefixRegEx)-1):
        if isAnOperator(prefixRegEx[index]):
            operators.append(prefixRegEx[index])
        if prefixRegEx[index] == " ":
            if isAUnaryOp(topElement(operators)):
                operator = operators.pop()
                if listIsEmpty(operators):
                    divided_expression = (prefixRegEx[index+1:], operator, None)
                    break
            else:
                if len(operators) > len(n_tokens):
                    n_tokens.append(index)
                else:
                    token_1 = n_tokens.pop()
                    token_2 = index
                    operator = operators.pop()
                    if listIsEmpty(operators):
                        divided_expression =  (prefixRegEx[token_1+1:token_2], operator, prefixRegEx[token_2+1:])
                        break
    return divided_expression
print(dividePrefixRegEx(". * . | 0 . 1 * 0 1 * 0"))

('* . | 0 . 1 * 0 1', '.', '* 0')


# Simplifier of Prefix RegEx

In [99]:
def simplifyRegEx(kettleOp, emptySequence, nullSequence, unionOp, concatOp, prefixRegEx):
    if not isAnOperator(prefixRegEx[0]):
        return prefixRegEx
    dividedRegEx = dividePrefixRegEx(prefixRegEx)
    if isAUnaryOp(dividedRegEx[1]):
        regEx = None
        if dividedRegEx[0][0] == kettleOp:
            regEx = simplifyRegEx(kettleOp, emptySequence, nullSequence, unionOp, concatOp, dividedRegEx[0][2:])
            return dividedRegEx[1] + " " + regEx
        else:
            regEx = simplifyRegEx(kettleOp, emptySequence, nullSequence, unionOp, concatOp, dividedRegEx[0])
        if isAnEmptySequence(regEx):
            return emptySequence
        if isANullSequence(regEx):
            return emptySequence
        return dividedRegEx[1] + " " + regEx
    leftRegEx = simplifyRegEx(kettleOp, emptySequence, nullSequence, unionOp, concatOp, dividedRegEx[0])
    rightRegEx = simplifyRegEx(kettleOp, emptySequence, nullSequence, unionOp, concatOp, dividedRegEx[2])
    if dividedRegEx[1] == unionOp:
        if leftRegEx == rightRegEx:
            return leftRegEx
    if dividedRegEx[1] == concatOp:
        if isANullSequence(leftRegEx) or isANullSequence(rightRegEx):
            return nullSequence
        if isAnEmptySequence(leftRegEx):
            return rightRegEx
        if isAnEmptySequence(rightRegEx):
            return leftRegEx
    return dividedRegEx[1] + " " + leftRegEx + " " + rightRegEx

#  Create NDFA from Prefix RegEx

In [100]:
def createNDFA(prefixRegEx):
    dividedRegEx = dividePrefixRegEx(prefixRegEx)
    if dividedRegEx[1] == None and dividedRegEx[2] == None:
        return stateGraphOp(dividedRegEx[0])(dividedRegEx[0])
    else:
        if dividedRegEx[2] == None:
            unaryOperator = stateGraphOp(dividedRegEx[1])
            return unaryOperator(createNDFA(dividedRegEx[0]))
        else:
            binaryOperator = stateGraphOp(dividedRegEx[1])
            graphR = createNDFA(dividedRegEx[0])
            graphS = createNDFA(dividedRegEx[2])
            return binaryOperator(graphR, graphS)

# DFA Node

In [101]:
class DFANode:
    def __init__(self, nodeName, dfaNodeList):
        self.nodeName = nodeName
        self.dfaNodeList = dfaNodeList
        
    def setVariableValue(self, variableName, value):
        if variableName in vars(self).keys():
            vars(self)[variableName] = value

# DFA Automaton

In [102]:
class DFA:
    def __init__(self, language, states, initalState, acceptingStates, transitionTable):
        self.language = language
        self.initialState = initalState
        self.states = states
        self.acceptingStates = acceptingStates
        self.transitionTable = transitionTable

# From NDFA To DFA List

In [142]:
from toolz import curry
import functools as func

@curry
def eClosure(emptySequence, stateNode):
    currentVisitedNodes = []
    currentVisitedNodes.append(stateNode)
    closure = {stateNode}
    for element in currentVisitedNodes:
        if (element.leftNode != None and element.leftLabel == emptySequence and
            not element.leftNode in currentVisitedNodes):
            currentVisitedNodes.append(element.leftNode)
            closure.add(element.leftNode)
        if (element.rightNode != None and element.rightLabel == emptySequence and
            not element.rightNode in currentVisitedNodes):
            currentVisitedNodes.append(element.rightNode)
            closure.add(element.rightNode)
    return closure

def getNameStateSet(startStr, endStr, separator, stateSet):
    name = ""
    for element in stateSet:
        name = element.stateName + ", "+ name
    name = name[0: len(name)-2]
    return startStr + name + endStr

@curry
def getLambdaClosures(emptySequence, NDFAList):
    closures = []
    for node in NDFAList:
        closures.append((node, eClosure(emptySequence)(node)))
    return closures

@curry
def hasAcceptingState(eclosure, finalState):
    for state in eclosure:
        if state.stateName == finalState.stateName:
            return True
    return False

def getLambdaClosure(node, lambdaClosures):
    for state in lambdaClosures:
        if state[0].stateName == node.stateName:
            return state[1]
@curry
def getLambdaTransition(symbol, lambdaClosures, state):
    lambdaTransition = set()
    for node in state:
        if node.leftNode != None and node.leftLabel == symbol:
            lambdaTransition = lambdaTransition.union(getLambdaClosure(node.leftNode, lambdaClosures))
        if node.rightNode != None and node.rightLabel == symbol:
            lambdaTransition = lambdaTransition.union(getLambdaClosure(node.rightNode, lambdaClosures))
    return lambdaTransition

def lambdaTrasitionIncluded(lambdaTransition, states):
    for state in states:
        if state[0] == lambdaTransition:
            return True
    return False

def fromNDFAToDFAList(symbolSet, emptySequence, NDFAList, finalState):
    lambdaClosures = getLambdaClosures(emptySequence)(NDFAList)
    states = []
    if hasAcceptingState(lambdaClosures[0][1])(finalState):
        states.append((lambdaClosures[0][1], "F", []))
    else:
        states.append((lambdaClosures[0][1], "T", []))
    for state in states:
        for symbol in symbolSet:
            lambdaTransition = getLambdaTransition(symbol)(lambdaClosures)(state[0])
            if lambdaTransition != set():
                state[2].append((lambdaTransition, symbol))
                if not lambdaTrasitionIncluded(lambdaTransition, states):
                    if hasAcceptingState(lambdaTransition)(finalState):
                        states.append((lambdaTransition, "F", []))
                    else:
                        states.append((lambdaTransition, "T", []))
                            
    return states

# Functions to draw  DFA List

In [104]:
from graphviz import Digraph
import os
nameOfFile = "fsm"
fileFormat = "pdf"
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
diagraph = Digraph('finite_state_machine', filename='fsme', format= "pdf")
diagraph.attr(rankdir='LR', size='8,5')

def drawAcceptingStateList(diagraph, representation, nodeShape, stateNameList):
    diagraph.attr(representation, shape=nodeShape)
    for stateName in stateNameList:
        diagraph.node(stateName)

def drawEdgeList(diagraph, representation, nodeShape, edgeList):
    diagraph.attr(representation, shape=nodeShape)
    for element in edgeList:
        diagraph.edge(*element)

def drawFSM(diagraph, stateNameList, edgeList):
    drawAcceptingStateList(diagraph, "node", "doublecircle", stateNameList)
    drawEdgeList(diagraph, "node", "circle", edgeList)

# Window

In [143]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox

errorMessage = "Invalid Expression"

def getSymbolSet(prefixRegEx):
    symbolSet = set()
    flag = False
    expression = ""
    for symbol in prefixRegEx:
        if not isAnOperator(symbol) and symbol != " ":
            expression += symbol
        elif expression != "":
            symbolSet.add(expression)
            expression = ""
    return symbolSet

def createMainWindow(title, dimensions, resizable):
    window = tk.Tk()
    window.title(title)
    window.geometry(dimensions)
    window.resizable(*resizable)
    return window
    
def createButton(window, name, bWidth, bCommand):
    return ttk.Button(window, text=name, width=bWidth, command=bCommand)
        
def createEntry(window, txtVariable, eWidth):
    return ttk.Entry(window, textvariable=txtVariable, width=eWidth)

def show_error_message(message_content):
    tk.messagebox.showerror("Error", message_content)
    
def createDFAList(entry):
    if entry != None:
        string = entry.get()
        if theRegExIsValid(string):
            newString = string[0:len(string)-1]
            prefixRegEx = fromInfixToPrefix(openPar)(closePar)(newString)
            args = [kettleOp, emptySequence, nullSequence, unionOp, concatOp, prefixRegEx]
            prefixRegEx = simplifyRegEx(*args)
            symbolSet = getSymbolSet(prefixRegEx)
            NDFA = createNDFA(prefixRegEx)
            NDFANamedList = nameNDFANodesList(reduceStateGraphToList(NDFA.initialState))
            args = [symbolSet, emptySequence, NDFANamedList, NDFA.finalState]
            DFAList = fromNDFAToDFAList(*args)
            return None
        return None


def getDFADrawed():
        DFAList = createDFAList(entry)
        if DFAList != None:
            DFAEdgeList = getEdgeList(DFAList)
            DFAFinalStates = getAcceptanceStatesNameList(getAcceptanceStatesFromDFAList(DFAList))
            drawFSM(diagraph, DFAFinalStates, DFAEdgeList)
        else:
            show_error_message(errorMessage)
    
def visualizeDFA():
    getDFADrawed()
    diagraph.view()
        
title = "DFA"
dimensions = "460x25"
resizable = (False, False)
window = createMainWindow(title, dimensions, resizable)
txtVariable = ""
eWidth = 50
entry = createEntry(window, txtVariable, eWidth)
name = "Convert To DFA"
bWidth = 20
button = createButton(window, name, bWidth, visualizeDFA)
eCoordinates = (0, 0)
entry.place(x=eCoordinates[0], y=eCoordinates[1])
bCoordinates = (325, 0)
button.place(x=bCoordinates[0], y=bCoordinates[1])
window.mainloop()


In [138]:
s = set()
s.add(1)
t = (1, 2 ,3)
print(t[2])

3
